# Pandas 🐼🐼🐼

### Miglior strumento di data wrangling

Pandas è ideale per il data wrangling, ovvero il processo di trasformazione e pulizia dei dati grezzi per analisi e modelli, mentre NumPy/array eccelle nei calcoli intensivi una volta che il dataset è pronto.

| Esigenza tipica                                                  | Funzionalità Pandas                     | Quando serve                                                                |
| ---------------------------------------------------------------- | --------------------------------------- | --------------------------------------------------------------------------- |
| 1. **Importare** formati eterogenei (CSV, Excel, TSV, JSON, SQL) | `read_*()`                              | Ogni volta che i dati provengono da laboratori, EHR, registry o altri dispositivi |
| 2. **Pulire** dati sporchi o parziali                            | `dropna`, `fillna`, `astype`, `replace` | Prima di qualsiasi statistica o di usarli per Machine Learning                                          |
| 3. **Ristrutturare** il dataset (modificarne la struttura per facilitarne l’analisi o per adattarlo a specifiche esigenze)                                  | `melt`, `pivot`, `stack/unstack`        | Per allineare o sistemare misure longitudinali, matrici di espressione...                 |
| 4. **Filtrare & creare sottogruppi**                                    | Boolean indexing, `query`, `groupby`    | Focus su un fenotipo, un trattamento, un gene...                               |
| 5. **Aggregare & statistiche rapide**                            | `groupby().agg()`, `describe`           | Report clinici, cruscotti QC (Quality Control)...                                                |
| 6. **Merge/Join** di più fonti                                   | `merge`, `concat`                       | Integrare imaging + clinica + omics (Dati omici → Genomica, trascrittomica, proteomica, metabolomica)                                         |
| 7. **Time series per il monitoraggio continuo**                                               | `to_datetime`, `resample`, `rolling`    | Wearable, follow-up, Analisi di segnali continui in terapia intensiva (ICU waveform analysis)...                                           |


### Perché non basta usare liste Python o array NumPy al posto di Pandas?

| Esigenza tipica nei dati biomedici                     | Liste Python                                | NumPy array                                           | Pandas DataFrame                                |
| ------------------------------------------------------ | ------------------------------------------- | ----------------------------------------------------- | ----------------------------------------------- |
| **Etichette (ID paziente, nome gene, tempo)**          | ❌ gestite “a mano” con strutture ausiliarie | ❌ non esistono; solo indici interi                    | ✅ `index`, `columns`, gerarchici (`MultiIndex`) |
| **Gestire dati eterogenei** (numeri + stringhe + date)         | ✅ ma zero funzioni vettoriali               | ⚠️ possibile con `dtype=object`, ma perde in performance | ✅ colonne con tipi diversi, ottimizzate         |
| **Valori mancanti** (`NaN`, `None`, sentinel)          | ❌ serve logica ad-hoc                       | ⚠️ `np.nan` solo per float; complicato per tipi misti | ✅ `isna` per identificare valori mancanti, `fillna`per riempire valori mancanti, propagazione coerente per propagare il valore precedente o successivo nelle celle vuote.       |
| **Operazioni “SQL-like”** (`groupby`, `join`, `pivot`) | ❌ codice manuale e ciclo esplicito          | ❌ non previste                                        | ✅ una riga di codice idiomatica (ovvero con funzioni intuitive e efficienti)                |
| **Time series**             | ❌ da implementare                           | ⚠️ serve `np.searchsorted`/for loop                   | ✅ API dedicate (`resample` per aggregazione a intervalli temporali, `rolling` per applicare operazioni statistiche su una finestra mobile di dati, `shift` per confrontare valori tra istanti di tempo diversi) |
| **I/O (input/output) nativo** (CSV, Excel, SQL, Parquet)              | ❌ librerie esterne, parsing manuale         | ❌ idem                                                | ✅ `read_*`, `to_*` tutte funzioni one-liner                    |
| **Leggibilità notebook/paper**                         | 😐 spesso 3-4 strutture parallele           | 😐 array senza intestazioni esplicite                             | 😀 tabelle auto-formattate                      |


### 🏥 1 | Analisi di dataset clinici multifattoriali
(= studiare dati sanitari considerando più variabili contemporaneamente, come età, sesso, genetica, stile di vita, parametri fisiologici ...)

Scenario
Cartella clinica con: età, sesso, BMI, pressione S/D, HbA1c, diagnosi, outcome.

In [1]:
import pandas as pd
df = pd.read_csv("registry_diabete.csv")
df["diagnosi"] = df["diagnosi"].str.upper().str.strip()  # normalizzo
stats = (df.groupby("diagnosi")[["HbA1c", "pressione_S", "pressione_D"]]
           .mean()
           .round(2))

ModuleNotFoundError: No module named 'pandas'

Utilità:
- Pulizia nomenclatura diagnosi
- Statistiche descrittive per reparto
- Individuazione outlier (df[df["HbA1c"]>12])
- Esportazione rapida in Excel per il board clinico (stats.to_excel("summary.xlsx"))

{Tabella heat-map dei valori medi di HbA1c per diagnosi}

### 🧬 2 | Trascrittomica / RNA-seq
Scenario
Matrice genes × samples (>30 000 × 200).

In [ ]:
expr = pd.read_parquet("rna_seq.parquet")  # veloce su big data
# Filtra geni espressi
expr = expr.loc[expr.mean(axis=1) > 1]
corr = expr.T.corr(method="spearman")

Utilità:
- Lettura in Parquet → RAM-efficient
- Filtri basati su media/varianza
- Calcolo di reti di co-espressione
- Output diretto verso Seaborn/Scanpy

### 🧠 3 | Imaging e Quantificazione
Scenario
CSV export da 3D Slicer: ID paziente, ROI, volume (mm³), densità (HU).

In [ ]:
img = pd.read_csv("slicer_ROI_metrics.csv")
lesioni = img.query("volume_mm3 > 500 & ROI == 'tumor'")
trend = (lesioni.groupby("patient_id")
                 .apply(lambda g: g.sort_values("scan_date")
                                    .assign(delta=g["volume_mm3"].diff())))


Utilità:
- Filtraggio rapido su soglia volumetrica
- Calcolo delta-volume fra follow-up
- Facilita il merge con outcome clinici

### 🧫 4 | High-Content Screening (96-well, 384-well)
Scenario
Assorbanza OD a 0 h, 24 h, 48 h per 300 composti.

In [ ]:
raw = pd.read_excel("HCS_readout.xlsx")
pivot = raw.pivot_table(values="OD",
                        index=["compound_id"],
                        columns=["time_h"])
pivot["growth_%"] = (pivot[48] - pivot[0]) / pivot[0] * 100
top_hits = pivot.nlargest(10, "growth_%")

Utilità
- pivot_table rende il dataset “wide” per calcolo Δ
- Rank dei top-hits per successiva validazione
- Integrare annotazioni chimiche con merge

{Foto di piastra 384-well con pozzi colorimetrici + barplot crescita}

### 💉 5. Studi di Coorte Longitudinali
Scenario:
Dataset con pazienti seguiti per 5 anni, misurando outcome clinici (es. progressione di malattia, sopravvivenza, esiti).

In [ ]:
df["data_visita"] = pd.to_datetime(df["data_visita"])
df.sort_values(["paziente_id", "data_visita"], inplace=True)


cohort = pd.read_csv("coorte_cardiologia.csv", parse_dates=["visit_date"])
cohort.sort_values(["patient_id", "visit_date"], inplace=True)
cohort["years_from_baseline"] = (cohort["visit_date"] -
                                 cohort.groupby("patient_id")["visit_date"].transform("first")
                                ).dt.days / 365.25

Utilità:
- Ordinamento e calcolo di intervalli temporali
- Analisi per paziente o sottogruppo
- Calcolo della progressione (delta valori nel tempo)

{Grafico temporale con andamento di biomarcatori in un paziente nel tempo}

# Numpy

NumPy is a Python library that provides a simple yet powerful data structure: the n-dimensional array.

**Perché scegliere NumPy**
Pur conoscendo già Python “puro” (con i suoi cicli `for`, la lettura/scrittura di CSV, ecc.), NumPy introduce un paradigma che offre vantaggi concreti:

1. **Maggiore velocità**

   * NumPy utilizza algoritmi implementati in C, che eseguono operazioni in nanosecondi anziché in secondi.
2. **Riduzione dei cicli**

   * Grazie alle strutture array, è possibile comporre operazioni vettoriali che eliminano gran parte dei loop manuali e l’indice di iterazione.
3. **Codice più leggibile**

   * Senza cicli annidati, le espressioni nel codice assomigliano molto di più alle equazioni matematiche che si vogliono calcolare.
4. **Alta qualità e affidabilità**

   * Un’ampia comunità di sviluppatori mantiene NumPy veloce, di facile utilizzo e privo di bug.

**Conclusione**
Questi fattori hanno reso NumPy lo standard “de facto” per gli array multidimensionali in Python applicato alla data science. Molte librerie popolari si basano su NumPy: impararlo fornisce una solida base su cui poi sviluppare competenze più avanzate in ambiti specifici.


In [ ]:
%%bash
# attiviamo l'ambiente virutale
. .venv/bin/activate
# Installiamo numpy
pip install numpy

  Using cached numpy-2.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)


In [13]:
import numpy as np

CURVE_CENTER = 80 # Sintassi standard per le costanti
grades = np.array([72, 35, 64, 88, 51, 90, 74, 12])
grades_list = [72, 35, 64, 88, 51, 90, 74, 12]

print(grades)
print(grades_list)

[72 35 64 88 51 90 74 12]
[72, 35, 64, 88, 51, 90, 74, 12]


In [15]:
print(type(grades))
print(type(grades_list))

<class 'numpy.ndarray'>
<class 'list'>


In [14]:
def curve(grades):
     average = grades.mean()
     change = CURVE_CENTER - average
     new_grades = grades + change
     return np.clip(new_grades, grades, 100)

curve(grades)

array([ 91.25,  54.25,  83.25, 100.  ,  70.25, 100.  ,  93.25,  31.25])

1. **Importazione di NumPy**
   Alla riga 1 si importa la libreria NumPy con l’alias `np`, una convenzione che abbrevia i comandi successivi.

2. **Creazione dell’array**
   Alla riga 3 si definisce un array monodimensionale chiamato `grades` di lunghezza 8 e tipo `int64`. In seguito esplorerai forma e tipo dei dati più a fondo, ma per ora basti sapere che hai un vettore di 8 valori interi.

3. **Calcolo della media**
   Alla riga 5 si richiama il metodo `.mean()` sull’array: in un solo passaggio NumPy somma tutti gli elementi e ne restituisce la media. Gli array di NumPy dispongono di numerosi metodi analoghi per operazioni statistiche o matematiche.

4. **Vectorization e Broadcasting**
   Alla riga 7 si sfruttano due concetti chiave:

   * **Vectorization**: l’operazione (es. somma di uno stesso valore) viene applicata simultaneamente a tutti gli elementi dell’array, eliminando la necessità di cicli espliciti.
   * **Broadcasting**: NumPy “allinea” array di forme diverse per permettere calcoli vettoriali fra loro. Qui `grades` è un array shape `(8,)` e `change` è uno scalare shape `(1,)`; NumPy aggiunge automaticamente `change` a ciascun elemento di `grades`. Non avrebbe funzionato se grades fosse stata una lista!

5. **Clipping dei valori**
   Alla riga 8 si utilizza la funzione `np.clip()`, che garantisce che i voti “curve‑dati” non scendano sotto un minimo né superino un massimo.

   * Il secondo argomento di `clip()` è proprio l’array originale `grades`: così ogni voto corretto non scende mai al di sotto del suo valore iniziale.
   * Il terzo argomento è lo scalare `100`, che tramite broadcasting assicura che nessun voto ecceda il 100%.

> **Consiglio**: NumPy mette a disposizione decine di funzioni e metodi specializzati. Se ti sembra di ripetere operazioni comuni o di dover scrivere cicli, consulta la documentazione: molto probabilmente esiste già una routine adatta.


In [ ]:
%%bash
# installiamo matplotlib
pip install numpy matplotlib

## Forme degli array (Shape)

* **Shape** è la tupla che descrive la dimensione di ciascun asse di un array.
* Ogni array NumPy espone la proprietà `.shape`, che restituisce tale tupla.
* È fondamentale che gli array abbiano la forma che le funzioni si aspettano: un controllo rapido è stampare l’array insieme a `array.shape`.

**Esempio: creazione e verifica di un array 3D 2×2×3**

In [ ]:
import numpy as np

# Creo un vettore di 12 temperature e lo rimodello in un blocco 2×2×3
temperatures = np.array([
    29.3, 42.1, 18.8, 16.1, 38.0, 12.5,
    12.6, 49.9, 38.6, 31.3,  9.2, 22.2
]).reshape(2, 2, 3)
# Verifico la shape
print(temperatures.shape)

# Visualizzo l’array
print(temperatures)

(2, 2, 3)
[[[29.3 42.1 18.8]
  [16.1 38.  12.5]]

 [[12.6 49.9 38.6]
  [31.3  9.2 22.2]]]


In [ ]:
# Supponiamo di voler selezionare la temperatura 22.2
temperature = temperatures[1, 1, 2]
# ["tabella", "riga", "colonn"]
temperature

np.float64(22.2)

* Con tre o più dimensioni diventa difficile “vedere” i dati; per questo è utile

  * ribaltare (“swap”) assi con `.swapaxes()`
  * stampare shape e contenuto finché non si è certi della disposizione.

**Esempio: scambiare l’asse 1 con l’asse 2**

```python
reordered = np.swapaxes(temperatures, 1, 2)
print(reordered)
# Output:
# array([[[29.3, 16.1],
#         [42.1, 38. ],
#         [18.8, 12.5]],
#
#        [[12.6, 31.3],
#         [49.9,  9.2],
#         [38.6, 22.2]]])
```

---

## Assi (Axes)

* Gli **assi** indicano le dimensioni ed sono indicizzati da 0 in su.

  * In un array 2D, `axis=0` è l’asse verticale (righe), `axis=1` quello orizzontale (colonne).
* Molte funzioni NumPy cambiano comportamento a seconda dell’argomento `axis`.

**Esempio con `.max()`**

```python
import numpy as np

table = np.array([
    [5, 3, 7, 1],
    [2, 6, 7, 9],
    [1, 1, 1, 1],
    [4, 3, 2, 0],
])

# Massimo sull’intero array
print(table.max())        # 9

# Massimo per ciascuna colonna (asse 0)
print(table.max(axis=0))  # array([5, 6, 7, 9])

# Massimo per ciascuna riga (asse 1)
print(table.max(axis=1))  # array([7, 9, 1, 4])
```

* **Senza `axis`** → funzione su tutti i valori.
* **Con `axis=k`** → operazione lungo l’asse k, restituisce un array con dimensione ridotta di 1.

---

## Broadcasting

La regola fondamentale:

> Due array possono “broadcastarsi” se, per ogni asse, o hanno la stessa lunghezza, oppure almeno uno dei due è pari a 1.

* Se in un asse una dimensione vale 1, NumPy **duplica** quel dato lungo quell’asse.
* Se le dimensioni coincidono, operazione elemento-per-elemento.

**Esempio formale:**

* `A.shape = (4, 1, 8)`
* `B.shape = (1, 6, 8)`

| Asse   | A | B | Azione                 |
| ------ | - | - | ---------------------- |
| asse 0 | 4 | 1 | B duplicato 4 volte    |
| asse 1 | 1 | 6 | A duplicato 6 volte    |
| asse 2 | 8 | 8 | dimensioni uguali → OK |

**Creazione degli array**

```python
A = np.arange(32).reshape(4, 1, 8)
B = np.arange(48).reshape(1, 6, 8)
```

**Somma con broadcasting**

```python
C = A + B
print(C)
# Output:
# array([[[ 0,  2,  4,  6,  8, 10, 12, 14],
#         [ 8, 10, 12, 14, 16, 18, 20, 22],
#         [16, 18, 20, 22, 24, 26, 28, 30],
#         [24, 26, 28, 30, 32, 34, 36, 38],
#         [32, 34, 36, 38, 40, 42, 44, 46],
#         [40, 42, 44, 46, 48, 50, 52, 54]],
#
#        [[ 8, 10, 12, 14, 16, 18, 20, 22],
#         ...
#        ],
#        ...
# ])
```

* NumPy estende internamente A e B alle stesse dimensioni 4×6×8, poi somma elemento per elemento.

---

### Conclusione

* **Shape** e **axes** sono la base per navigare array multidimensionali.
* Controlla sempre `.shape` e, quando serve, usa `.swapaxes()` o funzioni con `axis=`.
* Comprendere il **broadcasting** permette di scrivere calcoli vettoriali puliti e veloci, evitando loop espliciti.
